<a href="https://colab.research.google.com/github/anhpdd/ml-property-valuation-klang-valley/blob/main/notebooks/2_2_Ridership_data_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Load and filter data
df = (
    pd.read_parquet("https://storage.data.gov.my/dashboards/prasarana_timeseries.parquet")
    .assign(date=lambda x: pd.to_datetime(x['date']))
    .query("date >= '2022-01-01'")
    .assign(year_month=lambda x: x['date'].dt.strftime('%Y-%m'))
)

# Aggregate monthly ridership
monthly_ridership = (
    df.groupby(['year_month', 'service', 'origin', 'destination'], as_index=False)
    ['passengers']
    .sum()
    .rename(columns={'year_month': 'date'})
    [['date', 'service', 'origin', 'destination', 'passengers']]
)

print(f"✅ Loaded {len(monthly_ridership):,} monthly ridership records")
monthly_ridership

,date,service,origin,destination,passengers
0,2023-01,rail,A0: All Stations,AG01: Sentul Timur,112237
1,2023-01,rail,A0: All Stations,AG02: Sentul,98256
2,2023-01,rail,A0: All Stations,AG03: Titiwangsa,95541
3,2023-01,rail,A0: All Stations,AG04: PWTC,141960
4,2023-01,rail,A0: All Stations,AG05: Sultan Ismail,40720
...,...,...,...,...,...
629335,2025-10,rail,SP31: Putra Heights,SP26: Taman Perindustrian Puchong,1912
629336,2025-10,rail,SP31: Putra Heights,SP27: Bandar Puteri,1760
629337,2025-10,rail,SP31: Putra Heights,SP28: Puchong Perdana,1930
629338,2025-10,rail,SP31: Putra Heights,SP29: Puchong Prima,4902


In [ ]:
# Filter for station aggregates
all_station = monthly_ridership.loc[
    (monthly_ridership['origin'] == 'A0: All Stations') |
    (monthly_ridership['destination'] == 'A0: All Stations')
].reset_index(drop=True)

# Outgoing dataframe
outgoing = (
    all_station
    .groupby(['origin', 'date'], as_index=False)
    ['passengers'].sum()
    .rename(columns={'origin': 'station_name', 'passengers': 'outgoing'})
)

# Incoming dataframe
incoming = (
    all_station
    .groupby(['destination', 'date'], as_index=False)
    ['passengers'].sum()
    .rename(columns={'destination': 'station_name', 'passengers': 'incoming'})
)

# Total ridership dataframe
ridership = (
    outgoing
    .merge(incoming, on=['station_name', 'date'], how='left')
    .fillna(0)
    .astype({'outgoing': int, 'incoming': int})
)

ridership

,station_name,date,outgoing
0,A0: All Stations,2023-01,12763943
1,A0: All Stations,2023-02,12322923
2,A0: All Stations,2023-03,14814863
3,A0: All Stations,2023-04,12329434
4,A0: All Stations,2023-05,14029921
...,...,...,...
5197,SP31: Putra Heights,2025-06,95582
5198,SP31: Putra Heights,2025-07,104364
5199,SP31: Putra Heights,2025-08,103758
5200,SP31: Putra Heights,2025-09,178167


In [ ]:
# Export dataset
#ridership.to_excel('/content/drive/MyDrive/Colab/Capstone 1/ridership_data.xlsx')